# 08: Delegáti a event
**autor: Erik Král ekral@utb.cz**

---


Obsah:
- Typ delegate
- Šablony delegátů Action, Predicate a Func
- Event

## Typ delegate

Delegát je uživatelsky definovaný typ představující jednu nebo více referencí na metody s konkrétním návratovým typem a parametry. Instanci delegáta můžeme tedy přiřadit reference na metody a tyto metody potom prostřednictvím této instance zavolat (říkáme také vyvolat - invoke). Pomocí delegátů můžeme předávat reference na metody jako argumenty jiným metodám.

Nejprve si deklarujeme typ delegáta, což je prakticky hlavička metody s klíčovým slovem `delegate`.

In [ ]:
delegate void MujDelegat(int x);

Delegát je referenční typ a proměnnou typu delegát vytvoříme následujícím způsobem. Pokud jí přiřadíme hodnutu `null`, tak nebude mít zatím referenci na žádnou metodu.

In [ ]:
MujDelegat d = null;

Nyní si definujeme metodu `Vypis`, kterou budeme používat v dalších příkladech

In [ ]:
void Vypis(int x)
{
    Console.WriteLine(x);
}

Proměnné `d` teď přiřadíme referenci na metodu `Vypis`, která stejně jako typ `MujDelegat` má navratový typ `void` a jeden parametr typu `int`. 

In [ ]:
MujDelegat d = Vypis;

Předchozí příkaz by bylo možné také zapsat delším způsobem. Oba zápisy jsou rovnocenné.

In [ ]:
MujDelegat d = new MujDelegat(Vypis);

Prostřednictvím proměnné `d` teď můžeme vyvolat metodu `Vypis`. Nejkratší výraz pro vyvolání metody je `d(3)`, který odpovídá výrazu `d.Invoke(3)`, častěji ale používáme [Null-conditional operator](https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/operators/member-access-operators#null-conditional-operators--and-) `?.`, který zavolá metodu `Invoke`, pouze pokud proměnná `d` nemá hodnotu `null`. 

In [ ]:
d(3); 
d.Invoke(3); 
d?.Invoke(3);

V předchozím příkazu jsme proměnné `d` přiřadili referenci jen na jednu metodu. S pomocí operátoru `+=` můžeme přiřadit proměnné `d` reference na více metod. Příkaz `d?.Invoke(3);` potom zavolá jak metodu `VypisA` tak metodu `VypisB`.

In [ ]:
void VypisA(int x)
{
    Console.WriteLine($"A {x}");
}

void VypisB(int x)
{
    Console.WriteLine($"B {x}");
}

In [ ]:
MujDelegat d = null;

d += VypisA;
d += VypisB;

d?.Invoke(3);

Operátor `-=` potom referenci z proměnné `d` odstraní. V kódu je potřeba dávat pozor abychom nezapoměli odstraňovat reference na metody, pokud je již nepotřebujeme, protože reference na metody se neodstraňují automaticky.

In [ ]:
MujDelegat d = null;

d += VypisA;
d += VypisB;

d?.Invoke(2); // vypise A 2 a B 2

d -= VypisA;

d?.Invoke(3); // vypise jen B 3

## Šablony delegátů

Pro nejčastější typy delegátů jsou připraveny šablony delegátů. Proto není potřeba psát vlastní delegáty. 

### `Action`

Je delegát metody který může mít více parametrů a nevrací žádnou hodnotu (návratový typ je void). 

Například následující typ proměnné:

In [ ]:
Action<string> o1 = null;

Nahrazuje delší zápis s pomocí definice delegáta.

In [ ]:
delegate void Operace(string msg);

Operace o1 = null;


Pokud metoda nevrací žádnou hodnotu a nemá žádný parametr, tak použijeme typ `Action`, například:

In [ ]:
void VypisAhoj()
{
    Console.WriteLine("Ahoj");
}

Action action = VypisAhoj;
action.Invoke();

### `Predicate` 

Je delegát metody, která vrací vždy boolean a má jeden parametr.

Například následující typ proměnné:

In [ ]:
Predicate<int> p = null;

Nahrazuje delší zápis s pomocí definice delegáta.

In [ ]:
delegate bool Operace(int x);

Operace p = null;

### `Func` 

Je delegát metody, která vrací hodnotu a má více parametrů. Návratový typ je uveden jako poslední. Jde o nejobecnější z šablon delegátů. 

Například následující typ proměnné:

In [ ]:
Func<int,string,bool> f = null;

Nahrazuje delší zápis s pomocí definice delegáta.

In [ ]:
delegate bool Operace(int a, string b);

Operace f = null;

## Event

Event představuje implementace návrhového vzoru Observer v jazyce C#. 
Obsahuje dvě složky, zdroj události, tedy objekt ve kterém nastala událost a pozorovatele, což je objekt, který chce být notifikován v případě vzniku události. Událost může mít více pozorovatelů a pozorovatel může naslouchat více událostem. Typické použití událostí je v rámci grafického uživatelského rozhraní
např. signalizace stisku tlačítka.

**Event poskytuje na rozdíl od delegátu** zapouzdření, protože:

- event je možné vyvolat pouze uvnitř třídy, která jej definuje, dále 
- event není možné „vynulovat“ mimo třídu a 
- pozorovatel nemá možnost zjistit informace o dalších pozorovatelích.



Event definujeme pomocí klíčového slova `event` u fieldu typu delegate, například v následující třídě `Obchod` definujeme `event` `poplach` typu `Action`.

In [16]:
class Obchod
{
    public event Action poplach;

    public void Vloupani()
    {
        poplach?.Invoke();
    }
}

In [19]:
Obchod obchod = new Obchod();

Přihlášení k notifikaci eventu se provádí pomocí operátoru `+=`. K eventu se přihlašuje delegát (existující metoda, lambda výraz). Analogicky je možné provést odhlášení pomocí operátoru `-=` (instance může odhlásit pouze sama sebe, případně toho, o kom ví že se přihlásil). Operátor `+=` a `-=` lze použít i u delegáta, ale u eventu narozdíl od delegáta není možné použít operátor přiřazení `=` a tím odstranit ostatní pozorovatele. V následujícím příkladu se k notifikaci eventu poplach přihlásí dvě metody `VyjezdPolicie` a `VyjezdHasici` poté je uveden příklad na odhlášení notifikace.

In [21]:
void VyjezdPolicie()
{
    Console.WriteLine("Vyjizdi policie");
}

void VyjezdHasici()
{
    Console.WriteLine("Vyjizdi hasici");
}

obchod.poplach += VyjezdPolicie;
obchod.poplach += VyjezdHasici;

obchod.Vloupani();

obchod.poplach -= VyjezdPolicie;

Vyjizdi policie
Vyjizdi hasici


Následující kód nepůjde přeložit, protože event je možné vyvolat pouze uvnitř třídy, která jej definuje a event není možné „vynulovat“ mimo třídu.

In [ ]:
obchod.poplach.Invoke(); // error protože event je možné vyvolat pouze uvnitř třídy, která jej definuje
obchod.poplach = null;   // error protože event není možné „vynulovat“ mimo třídu

Error: (1,8): error CS0070: Událost Obchod.poplach se může zobrazovat jenom na levé straně výrazu += nebo -= (s výjimkou případu, kdy se používá z typu Obchod).
(2,8): error CS0070: Událost Obchod.poplach se může zobrazovat jenom na levé straně výrazu += nebo -= (s výjimkou případu, kdy se používá z typu Obchod).

Více se o eventech můžete dozvědět například zde:

[Events (C# Programming Guide). Microsoft Docs. 2022](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/events/)

---
## Cvičení

1. Úkol delegate

Definujte typ delegate pro metody `Soucet`, `Vypis` a `JeSude` a vytvorte ukázku jeho použití.

In [3]:
int Soucet(int x, int y)
{
    return x + y;
}

void Vypis(string text)
{
    Console.WriteLine(text);
}

bool JeSude(int x)
{
    return x % 2 == 0;
}

// Definujte typ delegate pro metodu Soucet a vytvorte ukazku pouziti typu delegate

// Definujte typ delegate pro metodu Vypis a vytvorte ukazku pouziti typu delegate

// Definujte typ delegate pro metodu JeSude a vytvorte ukazku pouziti typu delegate


2. Úkol Action, Func nebo Predicate

Místo vlastního definovaného typu delegate použijte šablonu `Action`, `Func` nebo `Predicate`.

In [4]:
int Soucet(int x, int y)
{
    return x + y;
}

void Vypis(string text)
{
    Console.WriteLine(text);
}

bool JeSude(int x)
{
    return x % 2 == 0;
}

// Odstrante nasledujici delegaty
delegate int MujDelegat1(int x, int y);
delegate void MujDelegat2(string text);
delegate bool MujDelegat3(int x);

int x = 2;
int y = 3;

MujDelegat1 d1 = Soucet; // A nahradte je pomoci Action, Func nebo Predicate
int vysledek1 = d1.Invoke(x, y);
Console.WriteLine(vysledek1);

MujDelegat2 d2 = Vypis; // A nahradte je pomoci Action, Func nebo Predicate
d2.Invoke("ahoj");

MujDelegat3 d3 = JeSude;  // A nahradte je pomoci Action, Func nebo Predicate
bool vysledek2 = d3.Invoke(2);
Console.WriteLine(vysledek2);


5
ahoj
True


3. Úkol event

Doplňte spravně klíčové slovo, tak aby označené řádky v klientském kódu nešly přeložit a šlo o návrhový vzor observer.

In [8]:
class NakupniCentrum
{
    public string Nazev { get; set; }
    public Action<string> poplach;

    public NakupniCentrum(string nazev)
    {
        Nazev = nazev;
    }

    public void Pozar()
    {
        poplach?.Invoke(Nazev);
    }
}

void VyjezdHasicu(string nazev)
{
    Console.WriteLine($"Jedeme hasit {nazev}");
}

void VyjezdPolicie(string nazev)
{
    Console.WriteLine($"Jedeme pomahat a chranit do {nazev}");
}


NakupniCentrum nakupniCentrum = new NakupniCentrum("Zlate Jablko");
nakupniCentrum.poplach += VyjezdHasicu;
nakupniCentrum.poplach += VyjezdPolicie;

nakupniCentrum.Pozar();

nakupniCentrum.poplach -= VyjezdPolicie;

// Doplnte spravne klicove slovo, aby nasledujici radky nesly prelozit a slo o navrhovy vzor observer

nakupniCentrum.poplach = VyjezdHasicu;  // Řádek nepůjde přeložit
nakupniCentrum.poplach.Invoke("UTB");  // Řádek nepůjde přeložit
nakupniCentrum.poplach = null;          // Řádek nepůjde přeložit


Jedeme hasit Zlate Jablko
Jedeme pomahat a chranit do Zlate Jablko
Jedeme hasit UTB
